# Song Recommendataion System, Powered by `Implicit`
In the downsampled regime, a laptop with sufficient RAM can train and run a recommendation model. That is what we are going to do now, before considering using the cloud. (See `cloud_train.py` for that.)

The package used here, [`implicit`](https://github.com/benfred/implicit), implements the same ALS algorithm as Spark; both cite the [same paper](https://ieeexplore.ieee.org/document/4781121). Therefore, any work done --- cross-validation, in particular --- with [`implicit`](https://github.com/benfred/implicit) can (hopefully) inform that performed using Spark ALS. It is much easier and possibly quicker to iterate models on a single machine. 

#### Executive summary of results
While the recommendation system is performing much better than random guessing (AUC >= 0.75), suggesting songs based on their "popularity" among all users may be just as --- if not more --- effective (AUC ~- 0.98). This is not entirely surprising, considering how trends come and go in (popular) music. Why/how are we using the AUC? See the end this notebook for cross-validation attempts and a detailed explanation.

In [ ]:
# The protagonist today
from implicit.als import AlternatingLeastSquares

# For creating utility matrix
import scipy.sparse as sparse

import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np

# Always beautify
import matplotlib.pyplot as plt
plt.style.use('ggplot')

## Read Data and turn into Sparse Matrix

In [2]:
# Load computed rating data
df = pd.read_csv("../data/rec/upload.csv", names=['uid', 'song_id', 'rating'])

In [3]:
df.head(5)

,uid,song_id,rating
0,12333,15249349,0.298602
1,1164092,4594934,0.152269
2,1863042,4090645,0.721931
3,1863042,7048351,0.400000
4,3629720,4681151,0.398724


In [4]:
users = np.sort(df.uid.unique())
songs = df.song_id.unique()
ratings = df.rating.values

# Get the associated row indices
rows = df.uid.astype(CategoricalDtype(categories=users)).cat.codes

# Get the associated column indices
cols = df.song_id.astype(CategoricalDtype(categories=songs)).cat.codes

# Floating point accuracy not likely a major concern; use float32 instead 64 to speed up computations
utility_sparse = sparse.csr_matrix((ratings, (rows, cols)), 
                                   shape = (len(users), len(ratings)), dtype="float32")

In [9]:
# Measure sparsity --- 99.9! Use of csr_matrix justified

matrix_size = utility_sparse.shape[0]*utility_sparse.shape[1] # Number of possible interactions in the matrix
num_ratings = len(ratings) # Number of items interacted with
sparsity = 100*(1 - (num_ratings/matrix_size))
print(sparsity)

99.99648765410417


## Create Train/Test Sets via Masking


Explain masking here

Function `make_train` adopted from the blog of [Jesse Steinweg-Woods](https://jessesw.com/Rec-System/`). Made minor edits to 
1. Remove unnecessary casting to `int` after ceil op
2. Use Python's built-in ceil instead of np.ceil; former is quicker when working on a single scalar

In [12]:
import random
from math import ceil

def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    
    num_samples = ceil(pct_test*len(nonzero_pairs)) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [13]:
# VERY sparse matrix to begin with; mask 0.10 instead of the 0.20 default
train, test, altered_users = make_train(utility_sparse, pct_test=0.10)

## Functions for Training and Cross-Validating

The original blog post by [Jesse Steinweg-Woods](https://jessesw.com/Rec-System/`) utilized AUC as the metric for model performance; in short, ... 

However, Steinweg-Woods did not implement cross-validation routines, which are necessary for choosing the 'best' rank, regularization strength, and number of ALS iterations in a production system. That --- is my contribution here.

In [16]:
'''
    J. S.-W.‘s FUNCTIONS FOR COMPUTING AUC

    - Bug fixes and optimizations by Everest Law, for correct indexing behavior
    - Optimization: Use getrow() instead of indexing, saves ~20 microseconds per op. Significant when there >= 10^4 users.
    - Optimization: Preallocate arrays storing AUCs
'''

from sklearn import metrics

def auc_score(predictions, test):
    '''
        This simple function will output the area under the curve using sklearn's metrics. 
        - predictions: your prediction output
        - test: the actual target result you are comparing to
    
        Returns AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)   

def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    store_auc = np.zeros(shape=(len(altered_users),), dtype='float32')
    popularity_auc = np.zeros_like(store_auc)
    
#     store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
#     popularity_auc = [] # To store popular AUC scores
    
    
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for ix, user in enumerate(altered_users): # Iterate through each user that had an item altered        
        training_row = training_set.getrow(user).toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        
        # Fixed line; see commented line for original
        pred = item_vecs.dot( user_vec )[zero_inds]
        #pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set.getrow(user).toarray()[0, zero_inds].reshape(-1)
        
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        
        store_auc[ix] = auc_score(pred, actual) # Calculate AUC for the given user and store
        popularity_auc[ix] = auc_score(pop, actual) # Calculate AUC using most popular and score
        
#         store_auc.append(auc_score(pred, actual)) 
#         popularity_auc.append(auc_score(pop, actual)) 
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [18]:
# EVEREST LAW'S OWN FUNCTIONS 

import itertools

def train_model(data, factors=40, regularization=0.1, iterations=30, alpha=15):
    '''
        Train a SINGLE model using matrix 'data', according to hyperparameters
        specified in func arguments
        
        ** ARGUMENTS **
        data: csr_sparse matrix containing ratings; each row is a user, each col is an item
        factors: rank of decomposition
        regularization: L2 reg factor
        iterations: run ALS this many times
        alpha:
        
        Returns trained model, and two sets of vectors (NumPy arrays)
    '''
    
    # Take transpose of `data` because ALS expects cols (not rows) to represent users
    model = AlternatingLeastSquares(factors=factors, regularization=regularization, iterations=iterations)    
    model.fit( data.T * alpha )
    
    return model, model.user_factors, model.item_factors

def cross_validate(training_set, test_set, altered_users, factors, regularizations, iterations, alphas):
    '''
        For each set of hyperparameters, evaluate model performance (AUC).
        Forms the cartesian product of all supplied hyperparameter values, and
        then train a model on each of combination. Returns
        
        ** ARGUMENTS **
        factors, regularizations, iterations, alphas: see train_model() for meaning;
        MUST BE ITERABLES so that itertools.product can combine them together.
        
        Returns:
    '''
    
    cv_results = []
    
    for cFactor, cReg, cIter, cAlpha in itertools.product(factors, regularizations, iterations, alphas):
        print("cFactor: {}, cReg: {}, cIter: {}, cAlpha: {}".format(cFactor, cReg, cIter, cAlpha))
        _, user_vecs, item_vecs = train_model(training_set, cFactor, cReg, cIter, cAlpha)
        
        rec_auc, most_popular_auc = calc_mean_auc(training_set, 
                                                  altered_users, [user_vecs, item_vecs], test_set)
                
        # Dict follows argument naming of train_model; facilitates reuse like this: train_model(**params)
        params = {"factors": cFactor, "regularization": cReg, "iterations": cIter, "alpha": cAlpha}
        cv_results.append((params, rec_auc, most_popular_auc))
        
        # CV might take forever; print latest outcome to monitor process 
        print(cv_results[-1])
        
    return cv_results

## Cross-Validate Matrix Factorization!

Here, I try to measure how well the recommendation system recalls songs masked from the training set. This can be thought of as a binary classification problem: the recommendation system generates confidence values for whether there will be activity between the masked pairs of users and songs; then, we calculate an **AUC** based on the confidence values and the test set. Remember that the test set is a replicate of the original data, but binarized into 1's (0's) where there are (aren't) ratings, i.e. activity.

As stated in [scikit-learn's documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score), the scale of the confidence values doesn't matter when calculating the AUC. Therefore, we can use the model's predictions as they are, without converting them to e.g. probability measures.

As a **baseline comparison**, I have also computed an AUC using the masked songs' "popularity" and the test set. "Popularity" of each song is calculated simply by summing the corresponding column in the original data. The more plays/downloads there were, the higher the "popularity". As it turns out, it is hard to beat this baseline where **AUC == 0.981**.

Last but not least, the performance of `implicit` is impressive indeed --- 2 seconds per iteration, which is much, much quicker than Spark on Google Cloud. Probably due to communication overhead of the latter? Seriousy recommend using `implicit` (which can be further sped up via CUDA!) when there isn't too much data.

In [ ]:
# factors = [10]
# regularizations = [0.01, 0.1, 1.0]
# iterations = [1]
# alphas = [0.1, 1.0, 10.0]

factors = [10, 20]
regularizations = [0.01, 0.1]
iterations = [5,10]
alphas = [0.1,1.0,10]

cv_results = cross_validate(train, test, altered_users, factors, regularizations, iterations, alphas)

cFactor: 10, cReg: 0.01, cIter: 5, cAlpha: 0.1


100%|██████████| 5.0/5 [00:09<00:00,  2.05s/it]


({'factors': 10, 'regularization': 0.01, 'iterations': 5, 'alpha': 0.1}, 0.789, 0.981)
cFactor: 10, cReg: 0.01, cIter: 5, cAlpha: 1.0


100%|██████████| 5.0/5 [00:10<00:00,  2.27s/it]


({'factors': 10, 'regularization': 0.01, 'iterations': 5, 'alpha': 1.0}, 0.816, 0.981)
cFactor: 10, cReg: 0.01, cIter: 5, cAlpha: 10


100%|██████████| 5.0/5 [00:12<00:00,  2.64s/it]


In [ ]:
print(cv_results)